# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 6 2022 9:43AM,248031,10,0086057295,ISDIN Corporation,Released
1,Oct 6 2022 9:43AM,248031,10,0086057298,ISDIN Corporation,Released
2,Oct 6 2022 9:43AM,248031,10,0086058702,ISDIN Corporation,Released
3,Oct 6 2022 9:43AM,248031,10,0086058733,ISDIN Corporation,Released
4,Oct 6 2022 9:43AM,248031,10,0086058841,ISDIN Corporation,Released
5,Oct 6 2022 9:43AM,248031,10,0086058798,ISDIN Corporation,Released
6,Oct 6 2022 9:43AM,248031,10,0086058856,ISDIN Corporation,Released
7,Oct 6 2022 9:43AM,248031,10,0086058859,ISDIN Corporation,Released
8,Oct 6 2022 9:43AM,248031,10,0086058862,ISDIN Corporation,Released
9,Oct 6 2022 9:43AM,248031,10,0086058855,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,248026,Released,9
20,248027,Released,1
21,248029,Executing,1
22,248030,Released,1
23,248031,Released,12


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248026,NaN,NaN,9.0
248027,NaN,NaN,1.0
248029,NaN,1.0,NaN
248030,NaN,NaN,1.0
248031,NaN,NaN,12.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247922,0.0,0.0,44.0
247926,13.0,0.0,2.0
247972,4.0,3.0,7.0
247983,0.0,12.0,2.0
247985,0.0,0.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247922,0,0,44
247926,13,0,2
247972,4,3,7
247983,0,12,2
247985,0,0,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247922,0,0,44
1,247926,13,0,2
2,247972,4,3,7
3,247983,0,12,2
4,247985,0,0,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247922,,,44
1,247926,13,,2
2,247972,4,3,7
3,247983,,12,2
4,247985,,,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 6 2022 9:43AM,248031,10,ISDIN Corporation
12,Oct 6 2022 9:42AM,247922,15,"Alliance Pharma, Inc."
56,Oct 6 2022 9:37AM,248030,10,"Nextsource Biotechnology, LLC"
57,Oct 6 2022 9:36AM,248029,22,"NBTY Global, Inc."
58,Oct 6 2022 9:33AM,248024,10,ISDIN Corporation
84,Oct 6 2022 9:31AM,248027,12,SugarBear
85,Oct 6 2022 9:31AM,248026,12,Hush Hush
94,Oct 6 2022 9:29AM,248023,12,Hush Hush
95,Oct 6 2022 9:21AM,248021,10,ISDIN Corporation
118,Oct 6 2022 9:01AM,248018,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 6 2022 9:43AM,248031,10,ISDIN Corporation,,,12
1,Oct 6 2022 9:42AM,247922,15,"Alliance Pharma, Inc.",,,44
2,Oct 6 2022 9:37AM,248030,10,"Nextsource Biotechnology, LLC",,,1
3,Oct 6 2022 9:36AM,248029,22,"NBTY Global, Inc.",,1,
4,Oct 6 2022 9:33AM,248024,10,ISDIN Corporation,,,26
5,Oct 6 2022 9:31AM,248027,12,SugarBear,,,1
6,Oct 6 2022 9:31AM,248026,12,Hush Hush,,,9
7,Oct 6 2022 9:29AM,248023,12,Hush Hush,,,1
8,Oct 6 2022 9:21AM,248021,10,ISDIN Corporation,,,23
9,Oct 6 2022 9:01AM,248018,10,ISDIN Corporation,,,28


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 6 2022 9:43AM,248031,10,ISDIN Corporation,12,,
1,Oct 6 2022 9:42AM,247922,15,"Alliance Pharma, Inc.",44,,
2,Oct 6 2022 9:37AM,248030,10,"Nextsource Biotechnology, LLC",1,,
3,Oct 6 2022 9:36AM,248029,22,"NBTY Global, Inc.",,1,
4,Oct 6 2022 9:33AM,248024,10,ISDIN Corporation,26,,
5,Oct 6 2022 9:31AM,248027,12,SugarBear,1,,
6,Oct 6 2022 9:31AM,248026,12,Hush Hush,9,,
7,Oct 6 2022 9:29AM,248023,12,Hush Hush,1,,
8,Oct 6 2022 9:21AM,248021,10,ISDIN Corporation,23,,
9,Oct 6 2022 9:01AM,248018,10,ISDIN Corporation,28,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 6 2022 9:43AM,248031,10,ISDIN Corporation,12,,
1,Oct 6 2022 9:42AM,247922,15,"Alliance Pharma, Inc.",44,,
2,Oct 6 2022 9:37AM,248030,10,"Nextsource Biotechnology, LLC",1,,
3,Oct 6 2022 9:36AM,248029,22,"NBTY Global, Inc.",,1,
4,Oct 6 2022 9:33AM,248024,10,ISDIN Corporation,26,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 6 2022 9:43AM,248031,10,ISDIN Corporation,12.0,NaN,NaN
1,Oct 6 2022 9:42AM,247922,15,"Alliance Pharma, Inc.",44.0,NaN,NaN
2,Oct 6 2022 9:37AM,248030,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
3,Oct 6 2022 9:36AM,248029,22,"NBTY Global, Inc.",NaN,1.0,NaN
4,Oct 6 2022 9:33AM,248024,10,ISDIN Corporation,26.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 6 2022 9:43AM,248031,10,ISDIN Corporation,12.0,0.0,0.0
1,Oct 6 2022 9:42AM,247922,15,"Alliance Pharma, Inc.",44.0,0.0,0.0
2,Oct 6 2022 9:37AM,248030,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
3,Oct 6 2022 9:36AM,248029,22,"NBTY Global, Inc.",0.0,1.0,0.0
4,Oct 6 2022 9:33AM,248024,10,ISDIN Corporation,26.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2728085,133.0,4.0,4.0
12,744076,11.0,0.0,0.0
15,495848,46.0,0.0,13.0
19,495997,8.0,12.0,0.0
22,248029,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2728085,133.0,4.0,4.0
1,12,744076,11.0,0.0,0.0
2,15,495848,46.0,0.0,13.0
3,19,495997,8.0,12.0,0.0
4,22,248029,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,133.0,4.0,4.0
1,12,11.0,0.0,0.0
2,15,46.0,0.0,13.0
3,19,8.0,12.0,0.0
4,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,133.0
1,12,Released,11.0
2,15,Released,46.0
3,19,Released,8.0
4,22,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,22
Status,,,,,
Completed,4.0,0.0,13.0,0.0,0.0
Executing,4.0,0.0,0.0,12.0,1.0
Released,133.0,11.0,46.0,8.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,22
0,Completed,4.0,0.0,13.0,0.0,0.0
1,Executing,4.0,0.0,0.0,12.0,1.0
2,Released,133.0,11.0,46.0,8.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,22
0,Completed,4.0,0.0,13.0,0.0,0.0
1,Executing,4.0,0.0,0.0,12.0,1.0
2,Released,133.0,11.0,46.0,8.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()